<a href="https://colab.research.google.com/github/yuan7220/ESS/blob/main/ESS_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install segment-geospatial

In [ ]:
import os
import geopandas as gpd
import leafmap
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff

In [ ]:
dallas = gpd.read_file('/content/Council_Boundaries.shp')

In [ ]:
dallas

In [ ]:
dallas.plot()

In [ ]:
m = leafmap.Map(center=[32.776, -96.797],
                zoom=10,
                layersControl=True,
               attributeControl = True
                )
m.add_basemap('SATELLITE')
m.add_gdf(dallas, layer_name='Dallas Council Districts')
m

In [ ]:
bbox= m.user_roi_bounds()

In [ ]:
m.user_roi_bounds()

In [ ]:
image = 'CBD_sample.tif'
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite', overwrite=True)

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name='CBD_sample.tif')
m

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    sam_kwargs=None,
)

In [ ]:
image = "CBD_sample.tif"
sam.generate(image, output="masks.tif", foreground=True, unique=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis='off', alpha=1, output= "annotations.tif")

In [ ]:
leafmap.image_comparison(
    "CBD_sample.tif",
    "annotations.tif",
    label1 = "Satellite",
    label2 = "Image Segmentation"
)

In [ ]:
m.add_raster("annotations.tif", alpha=0.5, layer_name="Masks")
m

In [ ]:
sam.tiff_to_vector("masks.tif", "masks.shp")

In [ ]:

m.add_shp("masks.shp")
m